In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/My Drive/Colab Notebook/code

## Import lib

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from models import Unet_2levels, Dunet_2levels, DVAE_refiner, DVAE
from utils import load_DRIVE, get_generator, log_gaussian, get_mix_coef, get_my_metrics, topo_metric
from PIL import Image
import os

# change accordingly whether you want to use GPU or CPU
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Set parammeter

In [3]:
# how to combine the loss terms L1 and L2, matters in models with refinement
MIX_COEFF_INIT  = 0.99
MIX_COEFF_DECAY = 0.005
MIX_COEFF_MIN   = 0.3
START_DECAY = 0

# set train parameters
EPOCHS = 150
STEPS_PER_EPOCH = 300   # number of mini-batches fed at each epoch
BATCH_SIZE = 16         # number of patches per mini-batch
PATCH_SIZE = 64         # side dimension of each patch
SEED = 7

# set the pipeline to be executed and hyperparameters
MODEL = 'DVAE_refiner'    # Unet, Dunet, DVAE_refiner
LOSS_TYPE = 'BCE'         # can be one of BCE, BCEw, FL
ZDIM = 100                # number of feature maps of the 3D encoding space (DVAE)
USE_PRETRAINED = True     # to use pretrained models instead of training

# picking a model according to the selection
if MODEL == 'Unet':
    model = Unet_2levels().to(DEVICE)
elif MODEL == 'Dunet':
    model = Dunet_2levels().to(DEVICE)
else:
    model = DVAE_refiner(ZDIM).to(DEVICE)
  
print("Model uses GPU: ", next(model.parameters()).is_cuda)

Model uses GPU:  True


## Loading
- Loading dataset: DRIVE
- Loading weight: if using pretrained model

In [0]:
# Load Drive dataset
dir_cur = os.getcwd()
train_X, train_Y, train_F, train_S, test_X, test_Y, test_F = load_DRIVE(PATCH_SIZE, dir_cur)

# load weight
if USE_PRETRAINED:
  path = dir_cur + '/weights/' + MODEL.lower() + '_' + LOSS_TYPE.lower() + '_weights.pth'
  if (os.path.exists(path)):
    model.load_state_dict(torch.load(path))
  else:
    print("Error Loading weight: not found")

# # print size and type of parameters of Model's layer
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

## Evaluation model

In [0]:
# get metrics for train and test sets
# print('Getting the evaluation metrics...')
model.eval()
train_segs = np.zeros(train_Y.shape)
test_segs = np.zeros(test_Y.shape)
train_infeasible = 0
train_wrong = 0
train_correct = 0
test_infeasible = 0
test_wrong = 0
test_correct = 0


In [6]:
with torch.no_grad():
  for i in range(train_X.shape[0]):
    img = torch.from_numpy(train_X[i,:,:]).unsqueeze(dim=0).unsqueeze(dim=0).float().to(DEVICE)
    gt = train_Y[i,:,:]

    if MODEL=='Unet':
        seg = model(img)
    elif MODEL=='Dunet':
        _, seg = model(img)
    else:
        _,_,_,_, seg = model(img, phase='test')
    a,b,c = topo_metric(gt, seg.cpu().numpy()[0,0], 0.5, 1000)

    train_infeasible += a
    train_wrong += b
    train_correct += c   

    train_segs[i] = seg

TypeError: ignored

In [0]:
with torch.no_grad():
  for i in range(test_X.shape[0]):
    img = torch.from_numpy(test_X[i,:,:]).unsqueeze(dim=0).unsqueeze(dim=0).float().to(DEVICE)
    gt = test_Y[i,:,:]

    if MODEL=='Unet':
      seg = model(img)
    elif MODEL=='Dunet':
      _, seg = model(img)
    else:
      _,_,_,_, seg = model(img, phase='test')

    a,b,c = topo_metric(gt, seg.cpu().numpy()[0,0], 0.5, 1000)

    test_infeasible += a
    test_wrong += b
    test_correct += c   

    test_segs[i] = seg


In [0]:
  train_auc, train_acc, train_sen, train_spe = get_my_metrics(train_segs,train_Y,train_F)
  test_auc, test_acc, test_sen, test_spe = get_my_metrics(test_segs,test_Y,test_F)

In [0]:
with torch.no_grad():
  img = torch.from_numpy(train_X[0,:,:]).unsqueeze(dim=0).unsqueeze(dim=0).float().to(DEVICE)
  gt = train_Y[0,:,:]
  _,_,_,_, seg = model(img, phase='test')
  a,b,c = topo_metric(gt, seg.cpu().numpy()[0,0], 0.5, 1000)